In [1]:
%matplotlib inline

import zipline
from zipline.api import future_symbol, set_commission, set_slippage, schedule_function, date_rules, time_rules, continuous_future, order_target
from zipline.finance.commission import PerTrade, PerContract
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage, VolatilityVolumeShare
import pyfolio as pf

from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

from IPython.display import display
import ipywidgets as widgets
out = widgets.HTML()
display(out)

HTML(value='')

In [2]:
"""
Model Settings
"""
starting_portfolio = 50000000
risk_factor = 0.0015
stop_distance = 3
breakout_window = 50
vola_window = 40
slow_ma = 80
fast_ma = 40
enable_commission = True
enable_slippage = True  

In [4]:
def report_result(context, data):
    context.months += 1
    today = zipline.api.get_datetime().date()
    # 연율화된 수익률 계산
    ann_ret = np.power(context.portfolio.portfolio_value / starting_portfolio, 12 / context.months) - 1
    
    out.value = """{} We have traded {} months and the annualized return is {:.2%}""".format(today, context.months, ann_ret)

def roll_futures(context, data):
    open_orders = zipline.api.get_open_orders()
    
    for held_contract in context.portfolio.positions:
        if held_contract in open_orders: 
            continue
        
        days_to_auto_close = (
            held_contract.auto_close_date.date() - data.current_session.date()
        ).days
        if days_to_auto_close > 5:
            continue
        
        # 연속 시계열
        continuation = continuous_future(
                held_contract.root_symbol, 
                offset=0, 
                roll='volume', 
                adjustment='mul'
                )
        
        continuation_contract = data.current(continuation, 'contract')
        
        if continuation_contract != held_contract:
            # 보유 계약 수 확인
            pos_size = context.portfolio.positions[held_contract].amount
            # 현재 포지션 청산
            order_target(held_contract, 0)
            # 새로운 포지션 진입
            order_target(continuation_contract, pos_size)     
            
def position_size(portfolio_value, std, point_value):
    target_variation = portfolio_value * risk_factor
    contract_variation = std * point_value
    contracts = target_variation / contract_variation
    return int(np.nan_to_num(contracts)) 
    
def initialize(context):
    """
    Cost Settings
    """
    if enable_commission:
        comm_model = PerContract(cost=0.85, exchange_fee=1.5)
    else:
        comm_model = PerTrade(cost=0.0)
        
    set_commission(us_futures=comm_model)
    
    if enable_slippage:
        slippage_model=VolatilityVolumeShare(volume_limit=0.2)
    else:
        slippage_model=FixedSlippage(spread=0.0)      
        
    set_slippage(us_futures=slippage_model)
    
    """
    Markets to trade
    """ 
    currencies = [
        'AD',
        'BP',
        'CD',
        'CU',
        'DX',
        'JY',
        'NE',
        'SF',
    ]
    agricultural = [
        'BL',
        '_C',
        'CT',
        'FC',
        'KC',
        'LR',
        'LS',
        '_O',
        '_S',
        'SB',
        'SM',
        '_W',
    ]
    nonagricultural = [
        'CL',
        'GC',
        'HG',
        'HO',
        'LG',
        'NG',
        'PA',
        'PL',
        'RB',
        'SI',
    ]
    equities = [
        'ES',
        'NK',
        'NQ',
        'TW',
        'VX',
        'YM',
    ]
    rates = [
        'ED',
        'FV',
        'TU',
        'TY',
        'US',
    ]
    
    markets = currencies + agricultural + nonagricultural + equities + rates
    context.universe = [
        continuous_future(market, offset=0, roll='volume', adjustment='mul')
            for market in markets
    ]
    
    context.highest_in_position = {market: 0 for market in markets} 
    context.lowest_in_position = {market: 0 for market in markets}    
    
    schedule_function(daily_trade, date_rules.every_day(), time_rules.market_close())
    
    context.months = 0    
    
    schedule_function(
        func=report_result,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    ) 
    
def analyze(context, perf):
    returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
    pf.create_returns_tear_sheet(returns, benchmark_rets=None)
    
def daily_trade(context, data):
    hist = data.history(
        context.universe, 
        fields=['close','volume'], 
        frequency='1d', 
        bar_count=250,
    )
    
    hist['trend'] = hist['close'].ewm(span=fast_ma).mean() > hist['close'].ewm(span=slow_ma).mean()    
    
    open_pos = {
        pos.root_symbol: pos 
        for pos in context.portfolio.positions
    } 
    
    for continuation in context.universe:
        root = continuation.root_symbol
        h = hist.xs(continuation, 2)
        std = h.close.diff()[-vola_window:].std()

        if root in open_pos:
            p = context.portfolio.positions[open_pos[root]]
            
            # Long position
            if p.amount > 0:
                if context.highest_in_position[root] == 0:
                    context.highest_in_position[root] = p.cost_basis
                else:
                    context.highest_in_position[root] = max(
                        h['close'].iloc[-1], context.highest_in_position[root]
                    ) 

                stop = context.highest_in_position[root] - (std  * stop_distance)
                if h.iloc[-1]['close'] < stop:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
                elif h['trend'].iloc[-1] == False:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
            
            # Shrot position
            else:
                if context.lowest_in_position[root] == 0:
                    context.lowest_in_position[root] = p.cost_basis
                else:
                    context.lowest_in_position[root] = min(
                        h['close'].iloc[-1], context.lowest_in_position[root]
                    )
                
                stop = context.lowest_in_position[root] + (std  * stop_distance)
                
                if h.iloc[-1]['close'] > stop:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0
                elif h['trend'].iloc[-1] == True:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0                         
        
        # 포지션 없음
        else:
            # Bull trend
            if h['trend'].iloc[-1]:
                if h['close'][-1] == h[-breakout_window:]['close'].max(): 
                    contract = data.current(continuation, 'contract')

                    contracts_to_trade = position_size(context.portfolio.portfolio_value, std, contract.price_multiplier)
                    
                    contracts_cap = int(h['volume'][-20:].mean() * 0.2)
                    contracts_to_trade = min(contracts_to_trade, contracts_cap)
                    
                    order_target(contract, contracts_to_trade)
            # Bear trend
            else: 
                if h['close'][-1] == h[-breakout_window:]['close'].min(): 
                    contract = data.current(continuation, 'contract')

                    contracts_to_trade = position_size(context.portfolio.portfolio_value, std, contract.price_multiplier)
                    
                    contracts_cap = int(h['volume'][-20:].mean() * 0.2)
                    contracts_to_trade = min(contracts_to_trade, contracts_cap)
                    
                    order_target(contract, -1 * contracts_to_trade)
    
    if len(open_pos) > 0:   
        roll_futures(context, data)

In [ ]:
start = datetime(2001, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2019, 1, 2, 8, 15, 12, 0, pytz.UTC)

perf = zipline.run_algorithm(
    start=start, end=end, 
    initialize=initialize, 
    analyze=analyze,
    capital_base=starting_portfolio,  
    data_frequency = 'daily', 
    bundle='futures_db' # 맞춤형 번들을 구축해야 함
    )